In [250]:
import numpy as np
import pandas as pd

In [251]:
class LogisticRegression:
    def __init__(self, learning_rate=0.1, iterations=100, porog=0.5):
        self.weigts = None
        self.m = 1
        self.n = 1
        self.learning_rate = learning_rate
        self.bias = 1
        self.n_estimators = iterations
        self.porog = porog

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))
    
    def fit(self, X_now, y_now):
        self.bias = 0
        y = np.array(y_now)
        X = np.array(X_now)
        self.m, self.n = X.shape
        self.weigts = np.zeros(self.n)
        for n in range(self.n_estimators):
            dot = np.dot(X, self.weigts) + self.bias
            predicted_y = self.sigmoid(dot)
            dw = (1 / self.m) * np.dot(X.T, (predicted_y - y))
            db = (1 / self.m) * np.sum(predicted_y - y)
            self.weigts -= self.learning_rate * dw
            self.bias -= self.learning_rate * db
    
    def predict(self, X):
        dot = np.dot(X, self.weigts) + self.bias
        y_pred = self.sigmoid(dot)
        y = [1 if i > self.porog else 0 for i in y_pred]
        return y_pred

In [252]:
class Bagging:
    def __init__(self, model, depth, l_r, iter):
        self.model_type = model
        self.depth = depth
        self.models = {}
        self.m = 1
        self.n = 1
        self.l_r = l_r
        self.iter = iter
    
    def fit(self, X, y):
        self.m, self.n = X.shape
        data = self.bootstrapp(X, y)
        for i in range(self.depth):
            x_i = data[i][0]
            y_i = data[i][1]
            self.models[str(i)] = self.model_type(learning_rate=self.l_r, iterations=self.iter)
            self.models[str(i)].fit(x_i, y_i)
    
    def predict(self, X):
        pred = np.zeros(self.m)
        answer_mod = []
        for i in range(self.depth):
            i_now = str(i)
            answer_mod.append(self.models[i_now].predict(X))
        for i in range(self.m):
            for j in range(self.depth):
                pred[i] += answer_mod[j][i]
        pred /= self.depth
        return pred
    
    def bootstrapp(self, X, y):
        datasets = []
        for _ in range(self.depth):
            answer_x = []
            answer_y = []
            l = list(range(self.m))
            choice = np.random.choice(l, size=self.m)
            for i in choice:
                answer_x.append(X[i])
                answer_y.append(y[i])
            datasets.append([answer_x, answer_y])
        return datasets

In [253]:
X = np.array([[1, 2, 4, 7, 8], [7, 2, 1, 2, 5], [3, 2, 1, 1, 4], [0, 2, 1, 8, 1], [2, 1, 0, 8, 5]])
y = np.array([0, 0, 1, 1, 1])

In [254]:
bag = Bagging(model=LogisticRegression, depth=5, l_r=0.005, iter=10500)
bag.fit(X, y)

In [255]:
bag.predict(X)

array([0.6081888 , 0.22458566, 0.96077199, 0.98145567, 0.79823742])